In [353]:
# GET INPUT: artist name
artist_name = "bts"

# GET INPUT: writing path
directory = "lyrics/" + artist_name

In [354]:
import urllib3
import re
import os
from bs4 import BeautifulSoup
from bs4 import Comment
urllib3.disable_warnings()

In [355]:
#some messy codes
# def url_search_wrapper(field):
#     url_wrapper_search = "https://www.melon.com/search/{}/index.htm?section=&searchGnbYn=Y&kkoSpl=Y&kkoDpType=&linkOrText=T&ipath=srch_form&q="
#     return url_wrapper_search.format(field)

# url_header_search_artist = url_search_wrapper("artist")
# url_header_search_song = url_search_wrapper("song")

url_header_search_artist = "https://www.melon.com/search/artist/index.htm?section=&searchGnbYn=Y&kkoSpl=Y&kkoDpType=&linkOrText=T&ipath=srch_form&q="

# note: retrieving from song 1 to 1000
#       orderBy: 최신순=ISSUE_DATE (default), 인기순=POPULAR_SONG_LIST 가나다순=SONG_NAME_WEB_LIST
url_header_song_paging = "https://www.melon.com/artist/songPaging.htm?startIndex=1&pageSize=1000&listType=A&orderBy=ISSUE_DATE&artistId="

url_header_song_detail = "https://www.melon.com/song/detail.htm?songId="

def get_soup_from_url(url):
    http = urllib3.PoolManager()
    url_result = http.request('GET', url)

    ## TODO: handle 404 etc.
    decoded_html = url_result.data.decode('utf8')
    soup = BeautifulSoup(decoded_html, "lxml")
    
    return soup

In [356]:
"""
artist_name -> artist_id
"""
soup = get_soup_from_url(url_header_search_artist + artist_name)
related_tags = soup.find_all(href=re.compile("goArtistDetail"), limit=1)
# for tag in related_tags:
#     print(tag)
artist_id = re.findall(r'\d{6}', related_tags[0]['href'])[0]
# print(artist_id)

In [357]:
"""
artist_id -> songs[id, title, album_id, album_title]
"""
soup = get_soup_from_url(url_header_song_paging + artist_id)
songs = {}
related_tags = soup.find_all("a", href=re.compile("playSong"))
for i, tag in enumerate(related_tags):
#     print(tag.contents[0])
#     print(re.findall(r'\d{8}', tag['href'])[1])
    songs[i] = {}
    songs[i]['id'] = re.findall(r',(\d+)', tag['href'])[0]
    songs[i]['title'] = tag.contents[0]
related_tags = soup.find_all("a", href=re.compile("goAlbumDetail"))
for i, tag in enumerate(related_tags):
    
    try:
        songs[i]['album_id'] = re.findall(r'\d+', tag['href'])[0]
        songs[i]['album_title'] = tag.contents[0]
    except:
        ## TODO: inform user if a song is not playable (e.g. hidden track)
#         print(tag)
        continue

In [ ]:
"""
song_id -> lyric
"""
for i in songs:
    song_id = songs[i]['id']
    
    # filename issue workaround
    song_title = songs[i]['title'].replace(r'`', "'").replace(r': ', '：').replace(r'/', ':')
    album_title = songs[i]['album_title'].replace(r'`', "'").replace(r': ', '：').replace(r'/', ':')

    soup = get_soup_from_url(url_header_song_detail + song_id)
    related_tags = soup.find(class_="lyric")
    try:
        # WARNING: something melon drops(?) connection because of too much traffic
        for child in related_tags.children:
            if isinstance(child, Comment):
                child.extract()
        lyric_cleaned = related_tags.get_text("\n", strip=True)
        
        filepath = "{}/{}".format(directory, album_title)
        if not os.path.exists(filepath):
            os.makedirs(filepath)
        filename = "{}/{}.txt".format(filepath, song_title)
        with open(filename, 'w') as outfile:
            outfile.write(lyric_cleaned)
    except:
        ## TODO: inform user if the lyric is missing (e.g. japanese version)
        print(url_header_song_detail + song_id)
        continue

https://www.melon.com/song/detail.htm?songId=30995596
https://www.melon.com/song/detail.htm?songId=30995607
https://www.melon.com/song/detail.htm?songId=30637986
https://www.melon.com/song/detail.htm?songId=5606797
